Surface PM 2.5 Concentrations (Annual) cit.031	Download from http://fizz.phys.dal.ca/~atmos/martin/?page_id=140 under All Composition PM2.5 - 0.01° × 0.01° w GWR adjustment file  [.asc.zip] from 1998 - 2015

Download to RW_Data/Rasters/cit_031_PM_2_5/ 

In [2]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3

In [3]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/cit_031_PM_2_5/"

s3_file1 = "cit_031_PM_2_5_2015.asc"


s3_key_orig1 = s3_folder + s3_file1
s3_key_edit1 = s3_key_orig1[0:-4] + "_edit.tif"


class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Define local file locations

In [4]:
local_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit.031/"

file_name1 = "GlobalGWR_PM25_GL_201501_201512-RH35-NoNegs.asc"

local_orig1 = local_folder + file_name1

orig_extension_length = 4 #4 for each char in .tif

local_edit1 = local_orig1[:-orig_extension_length] + "edit.tif" 


Use rasterio to reproject and compress

In [5]:
files = [local_orig1]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -999.9000244140625, 'width': 36000, 'height': 12470, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.01, 0.0, -180.005,
       0.0, -0.01, 69.85499999999999), 'tiled': False}


In [6]:
# Note - this is the core of Vizz's netcdf2tif function
def reproject(orig_name, edit_name):
    with rio.open(orig_name, 'r') as src:
        # This assumes data is readable by rasterio
        # May need to open instead with netcdf4.Dataset, for example

        profile = src.profile
        rows = profile["height"]
        columns = profile["width"]

        print(rows)
        print(columns)

        # Latitude bounds
        south_lat = -90
        north_lat = 90

        # Longitude bounds
        west_lon = -180
        east_lon = 180

        transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)

        # Profile

        no_data_val = -999.9000244140625
        target_projection = 'EPSG:4326'
        target_data_type = np.float64

        profile = {
            'driver':'GTiff', 
            'height':rows, 
            'width':columns, 
            'count':1, 
            'dtype':target_data_type, 
            'crs':target_projection, 
            'transform':transform, 
            'compress':'lzw', 
            'nodata': no_data_val
        }
        
        windows = src.block_windows()

        with rio.open(edit_name, "w", **profile) as dst:
            for ix, window in windows:
                if(ix[0]%100==0):
                    
                    print(ix)
                data = src.read(1, window=window)
                dst.write(data.astype(profile["dtype"]), 1, window=window)

In [7]:
reproject(local_orig1, local_edit1)


12470
36000
(0, 0)
(100, 0)
(200, 0)
(300, 0)
(400, 0)
(500, 0)
(600, 0)
(700, 0)
(800, 0)
(900, 0)
(1000, 0)
(1100, 0)
(1200, 0)
(1300, 0)
(1400, 0)
(1500, 0)
(1600, 0)
(1700, 0)
(1800, 0)
(1900, 0)
(2000, 0)
(2100, 0)
(2200, 0)
(2300, 0)
(2400, 0)
(2500, 0)
(2600, 0)
(2700, 0)
(2800, 0)
(2900, 0)
(3000, 0)
(3100, 0)
(3200, 0)
(3300, 0)
(3400, 0)
(3500, 0)
(3600, 0)
(3700, 0)
(3800, 0)
(3900, 0)
(4000, 0)
(4100, 0)
(4200, 0)
(4300, 0)
(4400, 0)
(4500, 0)
(4600, 0)
(4700, 0)
(4800, 0)
(4900, 0)
(5000, 0)
(5100, 0)
(5200, 0)
(5300, 0)
(5400, 0)
(5500, 0)
(5600, 0)
(5700, 0)
(5800, 0)
(5900, 0)
(6000, 0)
(6100, 0)
(6200, 0)
(6300, 0)
(6400, 0)
(6500, 0)
(6600, 0)
(6700, 0)
(6800, 0)
(6900, 0)
(7000, 0)
(7100, 0)
(7200, 0)
(7300, 0)
(7400, 0)
(7500, 0)
(7600, 0)
(7700, 0)
(7800, 0)
(7900, 0)
(8000, 0)
(8100, 0)
(8200, 0)
(8300, 0)
(8400, 0)
(8500, 0)
(8600, 0)
(8700, 0)
(8800, 0)
(8900, 0)
(9000, 0)
(9100, 0)
(9200, 0)
(9300, 0)
(9400, 0)
(9500, 0)
(9600, 0)
(9700, 0)
(9800, 0)
(9900, 0)


Upload orig and edit files to s3

In [8]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))

# Edit
s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
                         Callback=ProgressPercentage(local_edit1))

/Users/Max81007/Desktop/Python/Resource_Watch/Raster/cit.031/GlobalGWR_PM25_GL_201501_201512-RH35-NoNegsedit.tif  284065534 / 284065534.0  (100.00%)